In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics

In [25]:
df = pd.read_csv('/Users/pronnoygoswami/Documents/Virginia Tech/Fall 2018/Data Analytics/Project/Yelp/dataset/model_test.csv')
print(df.head)

<bound method NDFrame.head of          Unnamed: 0             business_id  cool        date  funny  \
0                 5  vgfcTvK81oD4r50NMjU2Ag     0  2011-02-25    0.0   
1                18  alG1fb1kl2vmT8s34jbbHg     1  2015-09-05    1.0   
2                21  u73j2VQ3TGWdMO-AG7MABw     0  2017-07-06    0.0   
3                28  7AlULGZI1pHt0imODsqdkg     0  2012-02-12    0.0   
4                31  cP8ErpZfRzdVR5Fvks-pnQ     0  2012-02-12    0.0   
5                36  7vxm0G4g857HzRqqpeu6bw     0  2016-11-18    0.0   
6                41  XgRDG9E_WLEbl8S-QUqQkQ     0  2015-11-05    0.0   
7                48  CtI7MMRF-arzGJxA1nJccQ     0  2016-06-18    0.0   
8                52  e9UsRb-fmza5ukU8Ftduew     1  2016-07-02    1.0   
9                53  nUaLFTfUqVKMFQonsWBnag     0  2016-06-11    0.0   
10               61  lqCJiHlxzRpuYt-u_XQUvg     0  2016-03-06    1.0   
11               66  9E1q2uEMd881wnruicNTUA     0  2016-07-02    0.0   
12               73  qwy9xdDuCwaVf

In [18]:
train = pd.read_csv('/Users/pronnoygoswami/Documents/Virginia Tech/Fall 2018/Data Analytics/Project/Yelp/dataset/model_train.csv', skipinitialspace=True, usecols = ['text', 'stars'])
test = pd.read_csv('/Users/pronnoygoswami/Documents/Virginia Tech/Fall 2018/Data Analytics/Project/Yelp/dataset/model_test.csv', skipinitialspace=True, usecols = ['text', 'stars'])

In [22]:
train = pd.get_dummies(train, columns=['stars'])

KeyError: "['stars'] not in index"

In [ ]:
train.head()

In [ ]:
test = pd.get_dummies(test, columns=['stars'])

In [ ]:
train['text'][10923]

In [ ]:
lens = train.text.str.len()
lens.mean(), lens.std(), lens.max(), lens.min()

In [ ]:
lens.hist();

In [ ]:
import re, string

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

In [ ]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [ ]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenize,
                     min_df = 3, max_df = 0.9, strip_accents = 'unicode', use_idf = 1,
                     smooth_idf = 1, sublinear_tf = 1)

In [ ]:
trn_term_doc = vec.fit_transform(train['text'])

In [ ]:
test_term_doc = vec.transform(test['text'])

In [ ]:
trn_term_doc, test_term_doc

In [ ]:
trn_term_doc.shape

In [ ]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1)/((y==y_i).sum()+1)

In [ ]:
x = trn_term_doc
test_x = test_term_doc

In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1, y) /pr(0,y))
    m = LogisticRegression(C=4, dual = True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [ ]:
label_cols = ['stars_1', 'stars_2', 'stars_3', 'stars_4', 'stars_5']

In [ ]:
preds = np.zeros((len(test), len(label_cols)))
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

In [5]:
p = pd.DataFrame(preds, columns = [['stars_1', 'stars_2', 'stars_3', 'stars_4', 'stars_5']])

NameError: name 'preds' is not defined

In [6]:
p.to_csv('nb_preds.csv', index = False)

NameError: name 'p' is not defined

In [7]:
metrics.accuracy_score(np.argmax(test[label_cols].values, axis = 1), np.argmax(preds, axis = 1))

NameError: name 'label_cols' is not defined